In [2]:
import re
import os
import json
import pandas as pd

In [3]:

data = pd.read_csv("data_processed/test.csv")
print(data)

ARCHIVOS PROCESADOS: 1 TOTAL: 17741
             id                                                url   price  \
0      58155817  /mercedes-benz-clase-c-c-cabrio-220-d-2p-diese...   32499   
1      58594130  /mazda-mx-5-active-18-2p-gasolina-2006-en-ciud...   10600   
2      58548465  /kia-sorento-diesel-2005-en-malaga-58548465-co...    4500   
3      58594037  /opel-cabrio-14-t-ss-excellence-2p-gasolina-20...   12450   
4      58707917  /porsche-911-turbo-coupe-2p-gasolina-2016-en-b...  134900   
...         ...                                                ...     ...   
17736  58286355  /volkswagen-golf-diesel-2003-en-salamanca-5828...    4500   
17737  58715158  /peugeot-2008-allure-bluehdi-81kw-110cv-5p-die...   17000   
17738  58266055  /subaru-forester-2.0i-hybrid-cvt-executive-plu...   35640   
17739  58090021  /toyota-urban-cruiser-diesel-2010-en-malaga-58...    7900   
17740  58736945  /renault-grand-modus-dynamique-15dci85-eco2-5p...    4950   

             km  year      

         id          creationDate                                            title                                                url  priceamount  pricefinancedAmount  pricetaxTypeId  pricehasTaxes  priceindicator_average  priceindicator_rank          sellername  sellerisProfessional sellercontractId  sellerpack_legacyId sellerpack_type  sellerratings_scoreAverage  sellerratings_commentsNumber      km  year  cubicCapacity  provinceIds0 mainProvince  locationprovinceIds_0 locationmainProvince  locationmainProvinceId  locationcityId locationcityLiteral  makeId  modelId  fuelTypeId fuelType  bodyTypeId  warrantyid  warrantymonths  isFinanced  isCertified  isProfessional         publishedDate  hasUrge  offerTypeid offerTypeliteral      phone environmentalLabel  drivenWheelsId contractId  packlegacyId packtype  transmissionTypeId detailad_id                                   detailad_title                               detailad_description                                       detailad_url